In [62]:
from google.cloud import bigquery
import pandas as pd

In [3]:
# Set Global Variables
gcp_project = "formula-1-wc-analytics"

In [8]:
# EXTRACT
# Lets ingest our prepped enriched view from GCP BigQuery - lets add an introductory passage on how we created the enriched
# We will use application default credentials locally instead of exporting a service account key for security reasons.
client = bigquery.Client(gcp_project)

In [44]:
sql = f"""
    SELECT year, date, raceId as race_id, name as grand_prix, circuit_ref, circuit_name, country, winner_driver_ref AS winner_driver, won_home_race, url
    FROM `{gcp_project}.curated_zone.v_1950_2024_results_enriched`
    ORDER BY date DESC
"""

df = client.query_and_wait(sql).to_dataframe() # Utilizes pandas
# df.head()

In [99]:
# CLEAN
# Clean data type from string to bool so we dont have to constantly convert in the future when we want to evaluate
df_cleaned = df # this will create a pointer to original dataframe, which is ok since we won't use raw df from now on
df_cleaned['won_home_race'] = df['won_home_race'].astype(bool) 
df_cleaned.head()

,year,date,race_id,grand_prix,circuit_ref,circuit_name,country,winner_driver,won_home_race,url
0,2024,2024-12-08,1144,Abu Dhabi Grand Prix,yas_marina,Yas Marina Circuit,UAE,norris,False,https://en.wikipedia.org/wiki/2024_Abu_Dhabi_G...
1,2024,2024-12-01,1143,Qatar Grand Prix,losail,Losail International Circuit,Qatar,max_verstappen,False,https://en.wikipedia.org/wiki/2024_Qatar_Grand...
2,2024,2024-11-23,1142,Las Vegas Grand Prix,vegas,Las Vegas Strip Street Circuit,United States,russell,False,https://en.wikipedia.org/wiki/2024_Las_Vegas_G...
3,2024,2024-11-03,1141,São Paulo Grand Prix,interlagos,Autódromo José Carlos Pace,Brazil,max_verstappen,False,https://en.wikipedia.org/wiki/2024_S%C3%A3o_Pa...
4,2024,2024-10-27,1140,Mexico City Grand Prix,rodriguez,Autódromo Hermanos Rodríguez,Mexico,sainz,False,https://en.wikipedia.org/wiki/2024_Mexico_City...


In [101]:
# TRANSFORM
# Lets do some transformations and aggregations to identify the average home wins per driver per grand prix
df_agg_country = df_cleaned.copy(deep=True) # Deep copying here so as to not change original cleaned df when drilling
df_agg_country = df_agg_country.groupby(['country']).agg(
    num_races = ('race_id', 'count'),
    num_unique_winners = ('winner_driver', 'nunique'),
    num_home_wins = ('won_home_race', 'sum'), # We can use sum here because dtype is bool, pythonic way to count bools is to sum (bools are 0 or 1)
    num_unique_home_winners = ('winner_driver', lambda x: (x[df.loc[x.index, 'won_home_race']].nunique())), # apply a mask to the winner_driver series and count unique based on the boolean series created by checking won_home_race
)
df_agg_country['avg_home_win_per_driver'] = round(df_agg_country['num_home_wins'] / df_agg_country['num_unique_home_winners'], 2)
df_agg_country = df_agg_country.sort_values(by='avg_home_win_per_driver', ascending=False).reset_index()
df_agg_country.head()

,country,num_races,num_unique_winners,num_home_wins,num_unique_home_winners,avg_home_win_per_driver
0,Germany,79,43,13,4,3.25
1,Netherlands,34,21,3,1,3.00
2,Spain,61,29,3,1,3.00
3,France,64,34,8,3,2.67
4,UK,80,42,31,12,2.58


In [156]:
# Lets identify # of home wins by driver
df_agg_driver = df_cleaned.copy(deep=True) # Deep copying here so as to not change original cleaned df when drilling
df_agg_driver = df_agg_driver.groupby(['winner_driver', 'country']).agg( # winner_driver and country are now part of Multiindex
    num_home_wins = ('won_home_race', 'sum'),
)
df_agg_driver = df_agg_driver.groupby(['country']).agg(
    max_home_wins = ('num_home_wins', 'max'),
    top_driver = ('num_home_wins', lambda x: 'none' if x.max() == 0 else x.idxmax()[0])
)
df_agg_driver.sort_values(by='max_home_wins', ascending=False).head()

,max_home_wins,top_driver
country,,
Germany,9,michael_schumacher
UK,9,hamilton
France,6,prost
Spain,3,alonso
Netherlands,3,max_verstappen


In [158]:
# Lets join the two aggregated dfs back together to get a full picture of the data of home race wins for each country
df_joined = pd.merge(df_agg_country, df_agg_driver, left_on='country', right_on='country', how='inner')
df_joined.head()

,country,num_races,num_unique_winners,num_home_wins,num_unique_home_winners,avg_home_win_per_driver,max_home_wins,top_driver
0,Germany,79,43,13,4,3.25,9,michael_schumacher
1,Netherlands,34,21,3,1,3.00,3,max_verstappen
2,Spain,61,29,3,1,3.00,3,alonso
3,France,64,34,8,3,2.67,6,prost
4,UK,80,42,31,12,2.58,9,hamilton


In [159]:
# Lets add another csv to the dataframe, country gdp...

In [160]:
# Visualize
# 1. Is there correlation between a Countries GDP and # of races hosted?
# 2. Countries ranked by # of home wins vs Countries ranked by avg home wins per driver